In [4]:
from chan import detect_fractals
import pandas as pd

df = pd.read_csv("/Users/cc/PycharmProjects/ChanLab/chan_data/BTCUSDT/5m_chanK.csv")

f_strict = detect_fractals(df, k=3, use_atr_normalize=True, strict=True)
print("Strict density:", len(f_strict)/len(df))

f_loose = detect_fractals(df, k=2, use_atr_normalize=True, strict=False)
print("Loose density:", len(f_loose)/len(df))


Strict density: 0.13063330875882898
Loose density: 0.27113605555423437


In [11]:
from pathlib import Path
import pandas as pd
from itertools import product
from chan import detect_fractals

ROOT = Path("/Users/cc/PycharmProjects/ChanLab")
DATA_CHAN_K = ROOT / "chan_data"

SYMBOLS = ["BTCUSDT", "ETHUSDT"]
TFS = ["5m", "15m", "30m"]

K_VALUES = [1, 2, 3]
MIN_AMPL_VALUES = [0.0, 0.001, 0.002, 0.005]
STRICT_VALUES = [True, False]
DEDUP_VALUES = [True, False]

def analyze_fractal_density(fractals_df, df_k):
    if fractals_df.empty or df_k.empty:
        return {}
    valid = fractals_df[fractals_df["is_valid"]]
    return {
        "total_fractals": len(fractals_df),
        "valid_fractals": len(valid),
        "top_fractals": int((valid["type"] == "top").sum()),
        "bottom_fractals": int((valid["type"] == "bottom").sum()),
        "fractal_density": len(valid) / len(df_k),
        "avg_strength": float(valid["strength"].mean()),
        "strength_std": float(valid["strength"].std(ddof=0)),
    }

def run_fractal_param_comparison(symbol, tf):
    src = DATA_CHAN_K / symbol / f"{tf}_chanK.csv"
    if not src.exists():
        print(f"[skip] {src}")
        return None

    df_k = pd.read_csv(src, parse_dates=["timestamp"])
    records = []

    for k, amp, strict, dedup in product(K_VALUES, MIN_AMPL_VALUES, STRICT_VALUES, DEDUP_VALUES):
        try:
            fr = detect_fractals(
                df_k,
                k=k,
                min_amplitude=amp,
                use_atr_normalize=True,
                enable_validation=strict,
                enable_dedup_run=dedup,
                strict=strict
            )
            stat = analyze_fractal_density(fr, df_k)
            if not stat:
                continue

            rec = dict(
                symbol=symbol, tf=tf,
                k=k, min_amp=amp,
                strict=strict, dedup=dedup,
                density=stat["fractal_density"],
                avg_strength=stat["avg_strength"],
                strength_std=stat["strength_std"],
                total=stat["total_fractals"],
                valid=stat["valid_fractals"],
                top=stat["top_fractals"],
                bottom=stat["bottom_fractals"],
                top_bottom_ratio=stat["top_fractals"] / max(1, stat["bottom_fractals"]),
            )
            records.append(rec)
        except Exception as e:
            print(f"[error] {symbol} {tf} k={k}, amp={amp}, strict={strict}, dedup={dedup}: {e}")

    return pd.DataFrame(records)

all_res = []
for s in SYMBOLS:
    for tf in TFS:
        df_res = run_fractal_param_comparison(s, tf)
        if df_res is not None:
            all_res.append(df_res)

summary = pd.concat(all_res, ignore_index=True)

print("[done] Parameter scanning is completed")
display(summary.head(20))

summary_group = (
    summary.groupby(["symbol", "tf", "strict", "dedup"])
    .agg(density_mean=("density", "mean"), strength_mean=("avg_strength", "mean"))
    .reset_index()
)
display(summary_group)


[done] 分型参数扫描完成（strict+dedup 双模式测试，未落盘）


symbol  tf  k  min_amp  strict  dedup   density  avg_strength  \
0   BTCUSDT  5m  1    0.000    True   True  0.443292      1.840156   
1   BTCUSDT  5m  1    0.000    True  False  0.443292      1.840156   
2   BTCUSDT  5m  1    0.000   False   True  0.443292      1.840156   
3   BTCUSDT  5m  1    0.000   False  False  0.443292      1.840156   
4   BTCUSDT  5m  1    0.001    True   True  0.443292      1.840156   
5   BTCUSDT  5m  1    0.001    True  False  0.443292      1.840156   
6   BTCUSDT  5m  1    0.001   False   True  0.443292      1.840156   
7   BTCUSDT  5m  1    0.001   False  False  0.443292      1.840156   
8   BTCUSDT  5m  1    0.002    True   True  0.443292      1.840156   
9   BTCUSDT  5m  1    0.002    True  False  0.443292      1.840156   
10  BTCUSDT  5m  1    0.002   False   True  0.443292      1.840156   
11  BTCUSDT  5m  1    0.002   False  False  0.443292      1.840156   
12  BTCUSDT  5m  1    0.005    True   True  0.443292      1.840156   
13  BTCUSDT  5m  1    0.005    True  False  0.443292      1.840156   
14  BTCUSDT  5m  1    0.005   False   True  0.443292      1.840156   
15  BTCUSDT  5m  1    0.005   False  False  0.443292      1.840156   
16  BTCUSDT  5m  2    0.000    True   True  0.201479      2.580794   
17  BTCUSDT  5m  2    0.000    True  False  0.235107      2.551475   
18  BTCUSDT  5m  2    0.000   False   True  0.271136      2.539511   
19  BTCUSDT  5m  2    0.000   False  False  0.271136      2.539511   

    strength_std  total  valid   top  bottom  top_bottom_ratio  
0       0.870081  18640  18640  9320    9320          1.000000  
1       0.870081  18640  18640  9320    9320          1.000000  
2       0.870081  18640  18640  9320    9320          1.000000  
3       0.870081  18640  18640  9320    9320          1.000000  
4       0.870081  18640  18640  9320    9320          1.000000  
5       0.870081  18640  18640  9320    9320          1.000000  
6       0.870081  18640  18640  9320    9320          1.000000  
7       0.870081  18640  18640  9320    9320          1.000000  
8       0.870081  18640  18640  9320    9320          1.000000  
9       0.870081  18640  18640  9320    9320          1.000000  
10      0.870081  18640  18640  9320    9320          1.000000  
11      0.870081  18640  18640  9320    9320          1.000000  
12      0.870081  18640  18640  9320    9320          1.000000  
13      0.870081  18640  18640  9320    9320          1.000000  
14      0.870081  18640  18640  9320    9320          1.000000  
15      0.870081  18640  18640  9320    9320          1.000000  
16      1.129509   8504   8472  4236    4236          1.000000  
17      1.150224   9918   9886  4956    4930          1.005274  
18      1.148823  11401  11401  5757    5644          1.020021  
19      1.148823  11401  11401  5757    5644          1.020021

symbol   tf  strict  dedup  density_mean  strength_mean
0   BTCUSDT  15m   False  False      0.305616       2.392937
1   BTCUSDT  15m   False   True      0.305616       2.392937
2   BTCUSDT  15m    True  False      0.281132       2.384096
3   BTCUSDT  15m    True   True      0.259366       2.413603
4   BTCUSDT  30m   False  False      0.308480       2.386765
5   BTCUSDT  30m   False   True      0.308480       2.386765
6   BTCUSDT  30m    True  False      0.279764       2.368171
7   BTCUSDT  30m    True   True      0.257423       2.393559
8   BTCUSDT   5m   False  False      0.303130       2.491776
9   BTCUSDT   5m   False   True      0.303130       2.491776
10  BTCUSDT   5m    True  False      0.279357       2.500842
11  BTCUSDT   5m    True   True      0.258048       2.523827
12  ETHUSDT  15m   False  False      0.316848       2.314397
13  ETHUSDT  15m   False   True      0.316848       2.314397
14  ETHUSDT  15m    True  False      0.291160       2.312352
15  ETHUSDT  15m    True   True      0.268353       2.342612
16  ETHUSDT  30m   False  False      0.320951       2.317817
17  ETHUSDT  30m   False   True      0.320951       2.317817
18  ETHUSDT  30m    True  False      0.291868       2.304634
19  ETHUSDT  30m    True   True      0.267103       2.339731
20  ETHUSDT   5m   False  False      0.311549       2.360797
21  ETHUSDT   5m   False   True      0.311549       2.360797
22  ETHUSDT   5m    True  False      0.286995       2.363916
23  ETHUSDT   5m    True   True      0.265055       2.391456

In [13]:
from pathlib import Path
import json
import pandas as pd
from chan import detect_fractals

ROOT = Path("/Users/cc/PycharmProjects/ChanLab")
DATA_CHAN_K = ROOT / "chan_data"
DATA_FRACTAL = ROOT / "chan_data" / "fractals"
CONFIG_PATH = ROOT / "configs/fractals_default.json"

PARAMS = json.loads(CONFIG_PATH.read_text())
print("[config loaded]")
print(json.dumps(PARAMS, indent=2, ensure_ascii=False))

SYMBOLS = ["BTCUSDT", "ETHUSDT"]
TFS = ["5m", "15m", "30m", "1h", "4h"]

for sym in SYMBOLS:
    for tf in TFS:
        src = DATA_CHAN_K / sym / f"{tf}_chanK.csv"
        if not src.exists():
            print(f"[skip] missing {src}")
            continue

        out_dir = DATA_FRACTAL / sym
        out_dir.mkdir(parents=True, exist_ok=True)
        out_file = out_dir / f"{tf}_fractals.csv"

        df_k = pd.read_csv(src, parse_dates=["timestamp"])
        fr = detect_fractals(df_k, **{k: v for k, v in PARAMS.items() if k != "mode"})

        fr.to_csv(out_file, index=False)
        print(f"[saved] {sym} {tf}: {len(fr)} fractals → {out_file.name}")

        density = len(fr) / len(df_k)
        valid = fr["is_valid"].sum() if "is_valid" in fr.columns else len(fr)
        print(f"    ├─ density={density:.3f}, valid={valid}/{len(fr)} ({valid/len(fr):.2%})")

print("\n[done] saved in  chan_data/fractals/")


[config loaded]
{
  "mode": "chan",
  "k": 2,
  "min_amplitude": 0.002,
  "use_atr_normalize": true,
  "strict": false,
  "enable_validation": false,
  "enable_dedup_run": true
}
[saved] BTCUSDT 5m: 11401 fractals → 5m_fractals.csv
    ├─ density=0.271, valid=11401/11401 (100.00%)
[saved] BTCUSDT 15m: 3871 fractals → 15m_fractals.csv
    ├─ density=0.272, valid=3871/3871 (100.00%)
[saved] BTCUSDT 30m: 1968 fractals → 30m_fractals.csv
    ├─ density=0.275, valid=1968/1968 (100.00%)
[saved] BTCUSDT 1h: 1004 fractals → 1h_fractals.csv
    ├─ density=0.282, valid=1004/1004 (100.00%)
[saved] BTCUSDT 4h: 242 fractals → 4h_fractals.csv
    ├─ density=0.290, valid=242/242 (100.00%)
[saved] ETHUSDT 5m: 11957 fractals → 5m_fractals.csv
    ├─ density=0.278, valid=11957/11957 (100.00%)
[saved] ETHUSDT 15m: 4013 fractals → 15m_fractals.csv
    ├─ density=0.282, valid=4013/4013 (100.00%)
[saved] ETHUSDT 30m: 1972 fractals → 30m_fractals.csv
    ├─ density=0.281, valid=1972/1972 (100.00%)
[saved] ET